# Chapter 6: Reccurrent Neural Networks (RNN)

Recurrent Neural Networks (RNN) can read inputs $x^{\langle t \rangle}$ one at a time, and remember some information/context through the hidden layer activations that get passed from one time-step to the next. This allows a uni-directional RNN to take information from the past to process later inputs (A bidirection RNN, outside the scope of the present notebook, can take context from both the past and the future). 

Simple, so-called 'Vanilla', RNNs are rarely used in practice due to the _vanishing gradient problem_. However, it is worthwhile to develop the simplest possible RNN model to understand the underlying concepts that also apply to LSTM and GRU architectures. To illustrate RNN predictions, we must use very short time series. We will use a character-level language model to generate mineral names from a dataset of existing mineral names (example of many-to-many RNN classification).

You will first build a Vanilla RNN from scratch, and second use an LSTM defined with Pytorch so that you can get a feeling of the data and model structure needed in more sophisticated applications. You will predict mineral names in both cases.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## 1. Mineral name dataset & character dictionary

We will first open the dataset of all known mineral names and then create the python dictionary `char_to_ix` (i.e., a _hash table_) to map each character to an index. We also create a second python dictionary `ix_to_char` that maps each index back to the corresponding character to later translate the probability distribution output of the softmax layer into the generated text.

<img src="figs_notebook/minerals.png" style="width:450;height:300px;">
<caption><center> Source: National Museums Scotland.</center></caption>

In [2]:
data = open('data/minerals.txt', 'r').read()
data = data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('The dataset contains %d total characters and %d unique characters.' % (data_size, vocab_size))
print('Unique characters are: ', sorted(chars))

char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print('Dictionary (index to character):', ix_to_char)
print('\nCharacter dataset excerpt (first 300 characters):')
data[:300]

The dataset contains 14631 total characters and 27 unique characters.
Unique characters are:  ['\n', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Dictionary (index to character): {0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}

Character dataset excerpt (first 300 characters):


'abelsonite\nabernathyite\nabhurite\nabramovite\nabswurmbachite\nacanthite\nachavalite\nactinolite\nacuminite\nadamite\nadelite\nadmontite\naegirine\naenigmatite\naerinite\naerugite\nafghanite\nafwillite\nagardite\nagrellite\nagrinierite\naguilarite\naheylite\nahlfeldite\naikinite\najoite\nakaganeite\nakatoreite\nakdalaite\naker'

The characters are a-z (26 characters) plus the "\n" (or newline) character, which represents the end of a mineral name.

## 2. RNN from scratch

Our RNN model will have the following structure: 

- Initialize parameters 
- Run the optimization loop
    - Forward propagation to compute the loss function
    - Backward propagation to compute the gradients with respect to the loss function
    - Clip the gradients to avoid exploding gradients (specific to RNNs)
    - Update parameter with gradient descent update rule
    - Sample characters to initiate new name generation
- Return learned parameters

The overall structure is similar to the one learned in chapter 4 for feedforward ANNs. The main difference is that the hidden layer is here a recursion on one cell, in time: At each time-step, the RNN tries to predict what is the next character given the previous characters. The dataset $X = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$ is a list of characters in the training set, while $Y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$ is such that at every time-step $t$, we have $y^{\langle t \rangle} = x^{\langle t+1 \rangle}$ (Fig. 1).

<img src="figs_notebook/rnn.png" style="width:450;height:300px;">
<caption><center> <b>Fig. 1</b>: Many-to-many RNN.</center></caption>

Another RNN specificity is _gradient clipping_ to avoid exploding gradients. Note also that to see the output of such a RNN model, we need to develop a sampling procedure that will generate new text $y$ based on previous text $x$. This will be detailed in a specific section below.

Figure 2 illustrates the operations for a single time-step of an RNN cell. 

<img src="figs_notebook/rnn_step_forward.png" style="width:700px;height:300px;">
<caption><center> <b>Fig. 2</b>: Vanilla RNN cell.</center></caption>
    
The cell takes as input $x^{\langle t \rangle}$ (current input) and $a^{\langle t - 1\rangle}$ (previous hidden state containing information from the past), and outputs $a^{\langle t \rangle}$ which is given to the next RNN cell and also used to predict $y^{\langle t \rangle}$ via the softmax function.

### 2.1. Parameter initialization

**EXERCISE 1:** initialize the model parameters with small random values using `np.random.randn(n1, n2)*0.01` for weights and `np.zeros((n, 1))` for biases.

In [3]:
def initialize_parameters(n_a, n_x, n_y):
    """
    Initialize parameters with small random values
    
    Returns:
    parameters -- python dictionary containing:
        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
        b --  Bias, numpy array of shape (n_a, 1)
        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    """
    np.random.seed(1)
    
    # YOUR CODE HERE
#    Wax = ?
#    Waa = ?
#    Wya = ?
#    b = ?
#    by = ?
    
    
    # SOLUTION
    Wax = np.random.randn(n_a, n_x)*0.01 # input to hidden
    Waa = np.random.randn(n_a, n_a)*0.01 # hidden to hidden
    Wya = np.random.randn(n_y, n_a)*0.01 # hidden to output
    b = np.zeros((n_a, 1))               # hidden bias
    by = np.zeros((n_y, 1))              # output bias
    
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b,"by": by}
    
    return parameters

### 2.2. Forward propagation

We will first define the RNN cell, and then the loop in time. Let us already define the `softmax` activation of the RNN (we will use `np.tanh` for the hyperbolic tangent function).

In [4]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

We are now going to implement the computation for a single time-step (i.e. RNN cell, see Fig. 2).

**EXERCISE 2**: Implement the RNN cell described in Figure 2.

**Instructions**:
1. Compute the hidden state with tanh activation: $a^{\langle t \rangle} = \tanh(W_{aa} a^{\langle t-1 \rangle} + W_{ax} x^{\langle t \rangle} + b_a)$.
2. Using your new hidden state $a^{\langle t \rangle}$, compute the prediction $\hat{y}^{\langle t \rangle} = softmax(W_{ya} a^{\langle t \rangle} + b_y)$.
3. Return $a^{\langle t \rangle}$ and $y^{\langle t \rangle}$

We will vectorize over $m$ examples. Thus, $x^{\langle t \rangle}$ will have dimension $(n_x,m)$, and $a^{\langle t \rangle}$ will have dimension $(n_a,m)$. _Hint:_ use `np.dot` for the dot product of two arrays.

In [5]:
#rename rnn_cell_forward
def rnn_step_forward(parameters, a_prev, x):
    """
    Implements a single forward step of the RNN-cell as described in Figure 2

    Arguments:
    xt -- your input data at timestep "t", numpy array of shape (n_x, m).
    a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        b --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    Returns:
    a_next -- next hidden state, of shape (n_a, m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    """

    # Retrieve parameters from "parameters"
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    b = parameters["b"]
    by = parameters["by"]

    # YOUR CODE HERE - compute next activation state using the formula given above
#    a_next = ?
    
    # SOLUTION
    a_next = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b)          # hidden state

    # YOUR CODE HERE - compute output of the current cell
#    p_t = ?
    
    # SOLUTION
    p_t = softmax(np.dot(Wya, a_next) + by)                             # probabilities for next characters
    
    # YOUR CODE HERE - SOLUTION
#    return ?, ?

    # SOLUTION
    return a_next, p_t

In [6]:
# sanity check
np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5,10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
b = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "b": b, "by": by}

a_next, yt_pred = rnn_step_forward(parameters, a_prev, xt)
print("a_next[4] = ", a_next[4])
print("a_next.shape = ", a_next.shape)
print("yt_pred[1] =", yt_pred[1])
print("yt_pred.shape = ", yt_pred.shape)

a_next[4] =  [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
a_next.shape =  (5, 10)
yt_pred[1] = [0.9888161  0.01682021 0.21140899 0.36817467 0.98988387 0.88945212
 0.36920224 0.9966312  0.9982559  0.17746526]
yt_pred.shape =  (2, 10)


**Expected Output**: 

<table>
    <tr>
        <td>
            **a_next[4]**:
        </td>
        <td>
           [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
        </td>
    </tr>
        <tr>
        <td>
            **a_next.shape**:
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **yt[1]**:
        </td>
        <td>
           [ 0.9888161   0.01682021  0.21140899  0.36817467  0.98988387  0.88945212
  0.36920224  0.9966312   0.9982559   0.17746526]
        </td>
    </tr>
        <tr>
        <td>
            **yt.shape**:
        </td>
        <td>
           (2, 10)
        </td>
    </tr>

</table>

We will now run `rnn_step_forward` $T_x$ times while updating the loss at each time increment $t$.

<img src="figs_notebook/rnn2.png" style="width:450;height:300px;">
<caption><center> <b>Fig. 3</b>: Many-to-many RNN with cell details.</center></caption>

**EXERCISE 3:** Complete the forward propagation of the RNN described in Figure 3, by updating the "next" hidden state and the cache by running `rnn_cell_forward`. _Hint:_ be careful with the time increments of the different inputs of the RNN cell.

In [7]:
def rnn_forward(X, Y, a0, parameters, vocab_size):
    """
    Implement the forward propagation of the recurrent neural network described in Figure (3).

    Arguments:
    x -- Input data for every time-step, of shape (n_x, m, T_x).
    a0 -- Initial hidden state, of shape (n_a, m)
    parameters -- python dictionary containing:
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        ba --  Bias numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)

    Returns:
    a -- Hidden states for every time-step, numpy array of shape (n_a, m, T_x)
    y_pred -- Predictions for every time-step, numpy array of shape (n_y, m, T_x)
    caches -- tuple of values needed for the backward pass, contains (list of caches, x)
    """

    x, a, y_hat = {}, {}, {}       # Initialize x, a and y_hat as empty dictionaries
    a[-1] = np.copy(a0)
    loss = 0                       # initialize your loss to 0
    
    for t in range(len(X)):
        
        # Set x[t] to be the one-hot vector representation of the t'th character in X.
        # if X[t] == None, we just have x[t]=0. This is used to set the input for the first timestep to the zero vector. 
        x[t] = np.zeros((vocab_size,1)) 
        if (X[t] != None):
            x[t][X[t]] = 1
        
        # YOUR CODE HERE - Run one step forward of the RNN
#        a[t], y_hat[t] = ?
        
        # SOLUTION
        a[t], y_hat[t] = rnn_step_forward(parameters, a[t-1], x[t])
        
        # Update the loss by substracting the cross-entropy term of this time-step from it
        loss -= np.log(y_hat[t][Y[t], 0])
        
    cache = (y_hat, a, x)
        
    return loss, cache

### 2.3. Backward propagation

When we implemented a simple (fully connected) neural network in chapter 4, we used backpropagation to compute the derivatives with respect to the cost to update the parameters. Similarly, in RNNs we can calculate the derivatives with respect to the cost in order to update the parameters. The backprop equations are quite complicated. They are only given below for completeness.

We will start by computing the backward pass for the Vanilla RNN-cell.

<img src="figs_notebook/rnn_cell_backprop.png" style="width:500;height:300px;"> <br>
<caption><center> <b>Fig. 4</b>: RNN-cell's backward pass.</center></caption>
    
Just like in a fully-connected neural network, the derivative of the cost function $J$ backpropagates through the RNN by following the chain-rule from calculus. The chain-rule is also used to calculate $(\frac{\partial J}{\partial W_{ax}},\frac{\partial J}{\partial W_{aa}},\frac{\partial J}{\partial b})$ to update the parameters $(W_{ax}, W_{aa}, b_a)$.

To compute the `rnn_step_backward`, we need to compute the following equations. The derivative of $\tanh$ is $1-\tanh(x)^2$. Note that: $ \text{sech}(x)^2 = 1 - \tanh(x)^2$

Similarly for $\frac{ \partial a^{\langle t \rangle} } {\partial W_{ax}}, \frac{ \partial a^{\langle t \rangle} } {\partial W_{aa}},  \frac{ \partial a^{\langle t \rangle} } {\partial b}$, the derivative of  $\tanh(u)$ is $(1-\tanh(u)^2)du$. 

The final two equations also follow the same rule and are derived using the $\tanh$ derivative. Note that the arrangement is done in a way to get the same dimensions to match.

In [8]:
def rnn_step_backward(dy, gradients, parameters, x, a, a_prev):
    """
    TO MODIFY - NOT MATCHING
    Implements the backward pass for the RNN-cell (single time-step).

    Arguments:
    da_next -- Gradient of loss with respect to next hidden state
    cache -- python dictionary containing useful values (output of rnn_cell_forward())

    Returns:
    gradients -- python dictionary containing:
                        dx -- Gradients of input data, of shape (n_x, m)
                        da_prev -- Gradients of previous hidden state, of shape (n_a, m)
                        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                        dba -- Gradients of bias vector, of shape (n_a, 1)
    """

    gradients['dWya'] += np.dot(dy, a.T)
    gradients['dby'] += dy
    da = np.dot(parameters['Wya'].T, dy) + gradients['da_next'] # backprop into h
    daraw = (1 - a * a) * da # backprop through tanh nonlinearity
    gradients['db'] += daraw
    gradients['dWax'] += np.dot(daraw, x.T)
    gradients['dWaa'] += np.dot(daraw, a_prev.T)
    gradients['da_next'] = np.dot(parameters['Waa'].T, daraw)
    return gradients

In [9]:
def rnn_backward(X, Y, parameters, cache):
    """
    MODIFY TEXT...
    Implement the backward pass for a RNN over an entire sequence of input data.

    Arguments:
    da -- Upstream gradients of all hidden states, of shape (n_a, m, T_x)
    caches -- tuple containing information from the forward pass (rnn_forward)
    
    Returns:
    gradients -- python dictionary containing:
                        dx -- Gradient w.r.t. the input data, numpy-array of shape (n_x, m, T_x)
                        da0 -- Gradient w.r.t the initial hidden state, numpy-array of shape (n_a, m)
                        dWax -- Gradient w.r.t the input's weight matrix, numpy-array of shape (n_a, n_x)
                        dWaa -- Gradient w.r.t the hidden state's weight matrix, numpy-arrayof shape (n_a, n_a)
                        dba -- Gradient w.r.t the bias, of shape (n_a, 1)
    """

    # Initialize gradients as an empty dictionary
    gradients = {}
    
    # Retrieve from cache and parameters
    (y_hat, a, x) = cache
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    
    # each one should be initialized to zeros of the same dimension as its corresponding parameter
    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['db'], gradients['dby'] = np.zeros_like(b), np.zeros_like(by)
    gradients['da_next'] = np.zeros_like(a[0])
    
    # Backpropagate through time
    for t in reversed(range(len(X))):
        dy = np.copy(y_hat[t])
        dy[Y[t]] -= 1
        gradients = rnn_step_backward(dy, gradients, parameters, x[t], a[t], a[t-1])
    
    return gradients, a

### 2.4. Gradient clipping

We will now implement the `clip` function that will be called between the backward pass and the parameter update. This will make sure that the RNN gradients are not "exploding" (i.e. overly large values). 

There are different ways to clip gradients; we will use a simple element-wise clipping procedure, in which every element of the gradient vector is clipped to lie between some range $[-N, N]$. More generally, you will provide a `maxValue` (say 10).

<img src="figs_notebook/clip.png" style="width:400;height:150px;">
<caption><center> <b>Fig. 5</b>: Visualization of gradient descent with & without gradient clipping, in a case where the network is running into slight "exploding gradient" problems. </center></caption>

**EXERCISE 4:** Implement the function below to return the clipped gradients of your dictionary `gradients`. Your function takes in a maximum threshold and returns the clipped versions of your gradients. Use `np.clip(a, a_min, a_max, out)`. `out` is the array where the results will be placed.

In [10]:
def clip(gradients, maxValue):
    '''
    Clips the gradients' values between minimum and maximum.
    
    Arguments:
    gradients -- a dictionary containing the gradients "dWaa", "dWax", "dWya", "db", "dby"
    maxValue -- clipping value
    
    Returns: 
    gradients -- a dictionary with the clipped gradients.
    '''
    
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    # YOUR CODE HERE: clip to mitigate exploding gradients, loop over [dWax, dWaa, dWya, db, dby]. (≈2 lines)

    
    
    # SOLUTION
    for gradient in [dWax, dWaa, dWya, db, dby]:
        np.clip(gradient, -maxValue, maxValue, out=gradient)

        

    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients

### 2.5. Parameter update

In [11]:
def update_parameters(parameters, gradients, learning_rate):

    parameters['Wax'] += -learning_rate * gradients['dWax']
    parameters['Waa'] += -learning_rate * gradients['dWaa']
    parameters['Wya'] += -learning_rate * gradients['dWya']
    parameters['b']  += -learning_rate * gradients['db']
    parameters['by']  += -learning_rate * gradients['dby']
    return parameters

### 2.6. Sampling

Now assume that your model is trained. You would like to generate new text (characters). The process of generation is explained in the picture below:

<img src="figs_notebook/sampling.png" style="width:500;height:300px;">
<caption><center> <b>Fig. 6</b>: In this picture, we assume the model is already trained. We pass in $x^{\langle 1\rangle} = \vec{0}$ at the first time step, and have the network then sample one character at a time. </center></caption>

Four steps are carried (all is already done for you). Still, read carefully about the process:

- **Step 1**: Pass the network the first "dummy" input $x^{\langle 1 \rangle} = \vec{0}$ (the vector of zeros). This is the default input before we've generated any characters. We also set $a^{\langle 0 \rangle} = \vec{0}$

- **Step 2**: Run one step of forward propagation to get $a^{\langle 1 \rangle}$ and $\hat{y}^{\langle 1 \rangle}$. Here are the equations:

$$ a^{\langle t+1 \rangle} = \tanh(W_{ax}  x^{\langle t \rangle } + W_{aa} a^{\langle t \rangle } + b)\tag{1}$$

$$ z^{\langle t + 1 \rangle } = W_{ya}  a^{\langle t + 1 \rangle } + b_y \tag{2}$$

$$ \hat{y}^{\langle t+1 \rangle } = softmax(z^{\langle t + 1 \rangle })\tag{3}$$

Note that $\hat{y}^{\langle t+1 \rangle }$ is a (softmax) probability vector (its entries are between 0 and 1 and sum to 1). $\hat{y}^{\langle t+1 \rangle}_i$ represents the probability that the character indexed by "i" is the next character.  We have provided a `softmax()` function that you can use.

- **Step 3**: Carry out sampling: Pick the next character's index according to the probability distribution specified by $\hat{y}^{\langle t+1 \rangle }$. This means that if $\hat{y}^{\langle t+1 \rangle }_i = 0.16$, you will pick the index "i" with 16% probability. To implement it, you can use [`np.random.choice`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.choice.html).

Here is an example of how to use `np.random.choice()`:
```python
np.random.seed(0)
p = np.array([0.1, 0.0, 0.7, 0.2])
index = np.random.choice([0, 1, 2, 3], p = p.ravel())
```
This means that you will pick the `index` according to the distribution: 
$P(index = 0) = 0.1, P(index = 1) = 0.0, P(index = 2) = 0.7, P(index = 3) = 0.2$.

- **Step 4**: The last step to implement in `sample()` is to overwrite the variable `x`, which currently stores $x^{\langle t \rangle }$, with the value of $x^{\langle t + 1 \rangle }$. You will represent $x^{\langle t + 1 \rangle }$ by creating a one-hot vector corresponding to the character you've chosen as your prediction. You will then forward propagate $x^{\langle t + 1 \rangle }$ in Step 1 and keep repeating the process until you get a "\n" character, indicating you've reached the end of the mineral name. 

In [12]:
def sample(parameters, char_to_ix, seed):
    """
    Sample a sequence of characters according to a sequence of probability distributions output of the RNN

    Arguments:
    parameters -- python dictionary containing the parameters Waa, Wax, Wya, by, and b. 
    char_to_ix -- python dictionary mapping each character to an index.
    seed -- used for grading purposes. Do not worry about it.

    Returns:
    indices -- a list of length n containing the indices of the sampled characters.
    """
    
    # Retrieve parameters and relevant shapes from "parameters" dictionary
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    # Step 1: Create the one-hot vector x for the first character (initializing the sequence generation). (≈1 line)
    x = np.zeros((vocab_size, 1))
    # Step 1': Initialize a_prev as zeros (≈1 line)
    a_prev = np.zeros((n_a, 1))
    
    # Create an empty list of indices, this is the list which will contain the list of indices of the characters to generate (≈1 line)
    indices = []
    
    # Idx is a flag to detect a newline character, we initialize it to -1
    idx = -1 
    
    # Loop over time-steps t. At each time-step, sample a character from a probability distribution and append 
    # its index to "indices". We'll stop if we reach 50 characters (which should be very unlikely with a well 
    # trained model), which helps debugging and prevents entering an infinite loop. 
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        
        # Step 2: Forward propagate x using the equations (1), (2) and (3)
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b)
        z = np.dot(Wya, a) + by
        y = softmax(z)
        
        # for grading purposes
        np.random.seed(counter+seed) 
        
        # Step 3: Sample the index of a character within the vocabulary from the probability distribution y
        idx = np.random.choice(list(range(vocab_size)), p = y.ravel())

        # Append the index to "indices"
        indices.append(idx)
        
        # Step 4: Overwrite the input character as the one corresponding to the sampled index.
        x = np.zeros((vocab_size, 1))
        x[idx] = 1
        
        # Update "a_prev" to be "a"
        a_prev = a
        
        # for grading purposes
        seed += 1
        counter +=1
        

    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

### 2.7. RNN model wrap-up & word generator

**EXERCISE 5:** Fill the `rnn_model` function that calls the functions doing the following: 

- Forward propagation to compute the loss function
- Backward propagation to compute the gradients with respect to the loss function
- Clip the gradients to avoid exploding gradients (specific to RNNs)
- Update parameter with gradient descent update rule

In [13]:
def rnn_model(X, Y, a_prev, parameters, vocab_size, learning_rate = 0.01):
    """
    Execute one step of the optimization to train the model.
    
    Arguments:
    X -- list of integers, where each integer is a number that maps to a character in the vocabulary.
    Y -- list of integers, exactly the same as X but shifted one index to the left.
    a_prev -- previous hidden state.
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        b --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    learning_rate -- learning rate for the model.
    
    Returns:
    loss -- value of the loss function (cross-entropy)
    gradients -- python dictionary containing:
                        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                        dWya -- Gradients of hidden-to-output weights, of shape (n_y, n_a)
                        db -- Gradients of bias vector, of shape (n_a, 1)
                        dby -- Gradients of output bias vector, of shape (n_y, 1)
    a[len(X)-1] -- the last hidden state, of shape (n_a, 1)
    """
    
    # YOUR CODE HERE
    # Forward propagate through time (≈1 line)
#    loss, cache = ?
    
    # Backpropagate through time (≈1 line)
#    gradients, a = ?
    
    # Clip your gradients between -5 (min) and 5 (max) (≈1 line)
#    gradients = ?
    
    # Update parameters (≈1 line)
#    parameters = ?
    
    
    # SOLUTION
    # Forward propagate through time (≈1 line)
    loss, cache = rnn_forward(X, Y, a_prev, parameters, vocab_size)
    
    # Backpropagate through time (≈1 line)
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    # Clip your gradients between -5 (min) and 5 (max) (≈1 line)
    gradients = clip(gradients, maxValue=5)
    
    # Update parameters (≈1 line)
    parameters = update_parameters(parameters, gradients, learning_rate)
    

    
    
    return loss, gradients, a[len(X)-1]

Given the dataset of mineral names, we use each line of the dataset (one name) as one training example. Every 100 steps of stochastic gradient descent, you will sample 10 randomly chosen names to see how the algorithm is doing. Remember to shuffle the dataset, so that stochastic gradient descent visits the examples in random order. The first entry of `X` being `None` will be interpreted by `rnn_forward()` as setting $x^{\langle 0 \rangle} = \vec{0}$. Further, this ensures that `Y` is equal to `X` but shifted one step to the left, and with an additional "\n" appended to signify the end of the mineral name. 

**EXERCISE 6:** Fill the gaps in `word_generator()`.

In [14]:
def word_generator(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, mineral_names = 7, vocab_size = 27):
    """
    Trains the model and generates mineral names. 
    
    Arguments:
    data -- text corpus
    ix_to_char -- dictionary that maps the index to a character
    char_to_ix -- dictionary that maps a character to an index
    num_iterations -- number of iterations to train the model for
    n_a -- number of units of the RNN cell
    mineral_names -- number of mineral names you want to sample at each iteration. 
    vocab_size -- number of unique characters found in the text, size of the vocabulary
    
    Returns:
    parameters -- learned parameters
    """
    
    # YOUR CODE HERE - Retrieve n_x and n_y from vocab_size
#    n_x, n_y = ?, ?
    
    
    # SOLUTION
    n_x, n_y = vocab_size, vocab_size

    
    # YOUR CODE HERE - Initialize parameters
#    parameters = initialize_parameters(?, ?, ?)
    
    
    # SOLUTION
    parameters = initialize_parameters(n_a, n_x, n_y)

    
    # Initialize loss (this is required because we want to smooth our loss, don't worry about it)
    seq_length = mineral_names
    loss = -np.log(1.0 / vocab_size) * seq_length
    
    # Build list of all mineral names (training examples).
    with open("data/minerals.txt") as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    

    np.random.seed(0)
    # YOUR CODE HERE - Shuffle list of all mineral names
#    np.random.shuffle(?)


    # SOLUTION
    np.random.shuffle(examples)
    
    
    # Initialize the hidden state of your RNN
    a_prev = np.zeros((n_a, 1))
    
    # Optimization loop
    for j in range(num_iterations):
        
        
        # Use the hint above to define one training example (X,Y) (≈ 2 lines)
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]] 
        Y = X[1:] + [char_to_ix["\n"]]
        
        # YOUR CODE HERE
        # Perform one optimization step: Forward-prop -> Backward-prop -> Clip -> Update parameters
        # Choose a learning rate of 0.01
#        curr_loss, gradients, a_prev = rnn_model(...)
        
        
        # SOLUTION
        curr_loss, gradients, a_prev = rnn_model(X, Y, a_prev, parameters, vocab_size, learning_rate = 0.01)
        
        
        # Use a latency trick to keep the loss smooth. It happens here to accelerate the training.
        loss = loss * 0.999 + curr_loss * 0.001

        # Every 2000 Iteration, generate "n" characters thanks to sample() to check if the model is learning properly
        if j % 2000 == 0:
            
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            # The number of mineral names to print
            seed = 0
            for name in range(mineral_names):
                
                # Sample indices and print them
                sampled_indices = sample(parameters, char_to_ix, seed)
                #print_sample(sampled_indices, ix_to_char)
                txt = ''.join(ix_to_char[ix] for ix in sampled_indices)
                txt = txt[0].upper() + txt[1:]  # capitalize first character 
                print ('%s' % (txt, ), end='')
                
                seed += 1  # To get the same result for grading purposed, increment the seed by one. 
      
            print('\n')
        
    return parameters

Run the following cell, you should observe your model outputting random-looking characters at the first iteration. After a few thousand iterations, the RNN model should learn to generate reasonable-looking mineral names. 

In [15]:
parameters = word_generator(data, ix_to_char, char_to_ix)

Iteration: 0, Loss: 23.087337

Nkzxwtdmfqoeyhsqwasjjjvu
Kneb
Kzxwtdmfqoeyhsqwasjjjvu
Neb
Zxwtdmfqoeyhsqwasjjjvu
Eb
Xwtdmfqoeyhsqwasjjjvu


Iteration: 2000, Loss: 23.513108

Liutordinite
Ite
Iusrodirite
Lacagoshabite
Xtordionitikite
Caadoshbanrite
Utoenhitelgorrarite


Iteration: 4000, Loss: 21.846217

Liusorgionite
Hled
Hwssohite
Lcabenphackomoite
Xtrofite
Caaglokbansite
Troelite


Iteration: 6000, Loss: 21.267887

Lhwtsselhopeuite
Ite
Iustogite
Lacaite
Xutrenglirthorr
Caadotfackrite
Utrengkite


Iteration: 8000, Loss: 20.922784

Lewtroginite
Hlacaite
Hytssengorevite
Lacagrolachsite
Wssrdinite
Baadrolachsite
Ussclerinternsarite


Iteration: 10000, Loss: 20.697659

Lewtrodite
Hifdaite
Hustlenenite
Lcaberrite
Wrroelite
Cabersincite
Ussengite


Iteration: 12000, Loss: 20.526648

Leytrocite
Erdaaepokacite
Guttoeite
Lacagroiicite
Wrrocite
Acagroiicite
Ussbiglinsite


Iteration: 14000, Loss: 20.485592

Inutorite
Epecaite
Evstilite
Ied
Vossenite
Acagriod
Tsscinite


Iteration: 16000, Loss: 20

**EXERCISE 7:** Interpret the RNN results by making observations about the evolution of results over training and how realistic the generated names look at the end (by comparing with patterns observed in the real mineral name database).

YOUR ANSWER HERE


SOLUTION

* We see that the words generated by the RNN are at first random. Very rapidly, realistic names emerge.
* Most existing mineral names end with 'ite' (have a look at the database we used as input). With such high probability, the RNN ends most of the generated words with 'ite'.

## 3. LSTM on Pytorch

We'll start off by importing the main PyTorch package and some important sub-libraries. We will then format the input data to solve the same problem as in section 2, but now with an LSTM model.

In [16]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.functional import F

import math
import string
import random

### 3.1. Input reformatting

Here, we need to reformat the data by ending each word with the "< EOS >" marker (meaning end of sequence/sentence).

In [17]:
def split_to_names(fname):
    EOS = "<EOS>"
    data = []
        
    with open(fname) as file:
        text = file.read().lower()
            
    names = text.splitlines()
    for i, name in enumerate(names):
        # Split names to chars and append the End of Sequence (EOS) Token
        ch_list = list(name) + [EOS]
        data.append(ch_list)
    return data

data_in_char = split_to_names("data/minerals.txt")

**EXERCISE 8:** Redefine the vocabulary `char_to_ix` and `ix_to_char` with the added special symbol. _Hint:_ We already did it in section 1.

In [18]:
# YOUR CODE HERE
char_vocab = ["<EOS>"] + sorted([ch for ch in string.ascii_lowercase])
#char_to_ix = ?
#ix_to_char = ?


# SOLUTION
char_to_ix = { ch:i for i,ch in enumerate(sorted(char_vocab)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(char_vocab)) }


Pytorch requires a special data format for loading inputs into its models. It is in two parts and involves first converting our data from strings (characters) to integers as our model understands only numbers. And the second part is the logic of how we will be loading our data to train our model. We will be using Pytorch’s `Dataset` and `DataLoader` class to handle both.

In [19]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_as_str, _map):
        self.data_as_int = []
        
        # Convert characters to integers
        for seq_as_str in data_as_str:
            seq_as_int = keys_to_values(seq_as_str, _map,
                random.choice(list(_map)))
            
            self.data_as_int.append(seq_as_int)

    def __len__(self):
        return len(self.data_as_int)

    def __getitem__(self, ix):
        # Get data sample at index ix
        item = self.data_as_int[ix]
        
        # Slice x and y from sample
        x = item[:-1]
        y = item[ 1:]
        return torch.tensor(x), torch.tensor(y)

def keys_to_values(keys, _map, default):
    return [_map.get(key, default) for key in keys]

Above, we first defined a Python class that inherits the Dataset class. This class will be responsible for fetching samples from our data. Next, we defined the `__init__` method, which takes in our already preprocessed data and converts each character to an integer using one of the dictionaries we created previously. Now our data is of a type our model can understand.
Pytorch’s Dataset class then requires that we define two other methods. These are the `__len__` method (which returns the length of our data), and the `__getitem__` method (which returns a sample at a particular index in our data). Implementing the `__len__` method is as simple as calling Python’s built-in function, `len`, on our data and returning the value. So that leaves us with the `__getitem__` method.
The `__getitem__` method receives the index we are interested in automatically as an argument. We then get the sample at that index in our data, slice out X (which is from the first value to the second value from the back) and Y (which is from the second value to the last value) from the sample, convert them to tensors and return them.

In [20]:
dataset = Dataset(data_in_char, char_to_ix)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

Lastly, we created a DataLoader object. This object provides a wrapper over our dataset that allows us to do (much easily) things like shuffling our dataset and more complex things like combining multiple datasets or using multiple workers to load our dataset. Luckily, we will not be dealing with any of the complex cases here. We finished by setting batch_size to 1 and shuffle to True.

### 3.2. LSTM model definition & training

LSTM model definition in Pytorch is shown below. It gives you an idea of the very different architecture and python code needed compared to Tensorflow 2 / Keras for example. Simply run the next cell to define your LSTM model.

In [21]:
class Model(nn.Module):
    def __init__(self, _map, hidden_size, emb_dim=8, n_layers=1):
        super(Model, self).__init__()
        
        self.vocab_size  = len(_map)
        self.hidden_size = hidden_size
        self.emb_dim     = emb_dim
        self.n_layers    = n_layers
        
        self.embedding = nn.Embedding(
            num_embeddings=self.vocab_size,
            embedding_dim =self.emb_dim)
        
        self.lstm = nn.LSTM(
            input_size =self.emb_dim,
            hidden_size=self.hidden_size,
            num_layers =self.n_layers,
            batch_first=True)
                
        self.fc = nn.Linear(
            in_features =self.hidden_size,
            out_features=self.vocab_size)
        
    def forward(self, x, prev_state):
        n_b, n_s = x.shape
        
        embed = self.embedding(x)
        yhat, state = self.lstm(embed, prev_state)
        
        out = self.fc(yhat)
        return out, state
    
    def init_state(self, b_size=1):
        return (torch.zeros(self.n_layers, b_size, self.hidden_size),
                torch.zeros(self.n_layers, b_size, self.hidden_size))

model = Model(char_to_ix, 64, 8, n_layers=1)

Now run the next cell to train the Pytorch LSTM model.

In [22]:
def train(model, data, num_iter, criterion, clip=0.25, lr=0.001, print_every=50):
    model.train()
    
    costs = []
    running_loss = 0
    optimizer = optim.Adam(model.parameters(), lr=lr)

    curr_iter = 0
    while curr_iter<num_iter:
        for x, y in data:
            optimizer.zero_grad()
            
            # Initialise model's state and perform forward-prop
            prev_state = model.init_state(b_size=x.shape[0])
            out, state = model(x, prev_state)

            # Calculate loss
            loss = criterion(out.transpose(1, 2), y)
            costs.append(loss.item())
            running_loss += loss.item()

            # Calculate gradients and update parameters
            loss.backward()
            if clip:
                nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
            
            curr_iter += 1
            if print_every and (curr_iter%print_every)==0:
                print("Iteration: {:{}}/{}, Loss: {:8.4f}".format(
                    curr_iter, int(math.log(num_iter, 10))+2, num_iter,
                    running_loss/float(print_every)))
                running_loss = 0
                
            if curr_iter>=num_iter:
                break
    return model, costs


criterion = nn.CrossEntropyLoss()
model, costs = train(
    model, dataloader, 35000, criterion, clip=5, lr=0.01, print_every=1000)

Iteration:   1000/35000, Loss:   1.9059
Iteration:   2000/35000, Loss:   1.7706
Iteration:   3000/35000, Loss:   1.7170
Iteration:   4000/35000, Loss:   1.6809
Iteration:   5000/35000, Loss:   1.6480
Iteration:   6000/35000, Loss:   1.6098
Iteration:   7000/35000, Loss:   1.5843
Iteration:   8000/35000, Loss:   1.5461
Iteration:   9000/35000, Loss:   1.5271
Iteration:  10000/35000, Loss:   1.4923
Iteration:  11000/35000, Loss:   1.4978
Iteration:  12000/35000, Loss:   1.4601
Iteration:  13000/35000, Loss:   1.4288
Iteration:  14000/35000, Loss:   1.4149
Iteration:  15000/35000, Loss:   1.4187
Iteration:  16000/35000, Loss:   1.3768
Iteration:  17000/35000, Loss:   1.3804
Iteration:  18000/35000, Loss:   1.3681
Iteration:  19000/35000, Loss:   1.3775
Iteration:  20000/35000, Loss:   1.3261
Iteration:  21000/35000, Loss:   1.3359
Iteration:  22000/35000, Loss:   1.3350
Iteration:  23000/35000, Loss:   1.3237
Iteration:  24000/35000, Loss:   1.2861
Iteration:  25000/35000, Loss:   1.2993


### 3.3. LSTM word generator

Similarly to the vanilla RNN, we need to apply a sampling method to generate new sequences of letters that will for our mineral names. This is done as follows:

In [23]:
def sample_next(model, x, prev_state, topk=5, uniform=True):
    # Perform forward-prop and get the output of the last time-step
    out, state = model(x, prev_state)
    last_out = out[0, -1, :]

    # Get the top-k indexes and their values
    topk = topk if topk else last_out.shape[0]
    top_logit, top_ix = torch.topk(last_out, k=topk, dim=-1)
    
    # Get the softmax of the topk's and sample
    p = None if uniform else F.softmax(top_logit.detach(), dim=-1).numpy()
    sampled_ix = np.random.choice(top_ix, p=p)
    return sampled_ix, state


def sample(model, seed, topk=5, uniform=True, max_seqlen=18, stop_on=None):
    seed = seed if isinstance(seed, (list, tuple)) else [seed]
    
    model.eval()
    with torch.no_grad():
        sampled_ix_list = seed[:]
        x = torch.tensor([seed])
        
        prev_state = model.init_state(b_size=1)
        for t in range(max_seqlen - len(seed)):
            sampled_ix, prev_state = sample_next(model, x, prev_state, topk, uniform)

            sampled_ix_list.append(sampled_ix)
            x = torch.tensor([[sampled_ix]])
            
            if sampled_ix==stop_on:
                break
    
    model.train()
    return sampled_ix_list

Run finally the next cell to generate some new mineral names. Of course, an LSTM model can be used on much longer temporal sequences. Here we simply showed how to use an LSTM instead of a vanilla RNN to predict mineral names.

In [24]:
for i in range(10):
    seed = random.choice(list(char_to_ix.values())[1:])
    print(seed, "=>", "".join(keys_to_values(
        sample(model, seed, 5, False, 30, char_to_ix["<EOS>"]),
        ix_to_char, "<?>")))

14 => nekucite<EOS>
10 => julinite<EOS>
10 => jishite<EOS>
13 => mellite<EOS>
15 => osmite<EOS>
15 => olivite<EOS>
23 => weimony<EOS>
10 => jurmosellite<EOS>
12 => lazleucite<EOS>
23 => weitgite<EOS>
